In [2]:
import socket
import time
from multiprocessing import Process, Value
from pynq.overlays.base import BaseOverlay

base = BaseOverlay("base.bit")
btns = base.btns_gpio

In [3]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
unsigned int write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    return 1;
}

//do not read here
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

In [22]:
#BUZZ FOR .5 SECONDS
def buzz():
    write_gpio(3, 0)
    time.sleep(.5)
    write_gpio(3, 1) #pull high
    
#-------------------------------------------------   
# SERVER PROCESS
def run_server(stop_flag):
    #'0.0.0.0' listens on any open IP automatically
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.bind(('0.0.0.0', 12345))
        except:
            sock.close
            sock.bind(('0.0.0.0', 12345))
        print ("listening...")
        sock.listen(1)

        while not stop_flag.value:
            try:
                sock.settimeout(1) #allow  to not hang on loop
                conn, addr = sock.accept()
                with conn:
                    print(f"Server: connected w/ {addr}")
                    while not stop_flag.value:
                        data = conn.recv(1024)
                        if not data:
                            break
                        if data.decode() == "BUZZ":
                            print("Server: Buzzing!")
                            buzz()
                        elif data.decode() == "SHUTDOWN":
                            print("Server: Shutting down!")
                            stop_flag.value = True
                            
            except socket.timeout:
                continue
        sock.close()
    
#------------------------------------------------- 
#CLIENT PROCESS
def run_client(target_ip, stop_flag):
    #BTN0 = CONNECT
    sock = None
    print("Client running. Button 0 to connect.")
    
    while not stop_flag.value:
        # BTN0 = connect
        if btns[0].read() == 1:
            try:
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.connect((target_ip, 12345))
                print("client connected")
                time.sleep(0.3) # debounce
            except:
                if sock:
                    sock.close()
                print("client: connection failed.")
                time.sleep(0.3)

        # BTN1 = send buzz
        if btns[1].read() == 1 and sock:
            try:
                sock.send("BUZZ".encode())
                print("Buzz!")
            except:
                print("client: no server connected")
            time.sleep(0.3)

        # BTN2 = disconnect and stop
        if btns[2].read() == 1:
            print("shutting down...")
            try:
                if sock:
                    sock.send("SHUTDOWN".encode())
                    sock.close()
            except:
                pass
            stop_flag.value = True # end loop
            time.sleep(0.3)

In [ ]:
if __name__ == "__main__":
    write_gpio(3, 1) #start low
    
    OTHER_IP = '192.168.0.120' #IP INPUT HERE
    stop_signal = Value('b', False) # shared flag to kill processes
    
    p1 = Process(target=run_server, args=(stop_signal,))
    p2 = Process(target=run_client, args=(OTHER_IP, stop_signal))
    
    p1.start()
    p2.start()
    
    p1.join()
    print("server shutdown")
    
    
    p2.join()
    print("client shutdown")
    
    print("DONE")

listening...
Client running. Button 0 to connect.
Server: connected w/ ('192.168.0.120', 51214)
Server: connected w/ ('192.168.0.120', 51218)
Server: Buzzing!
Server: Buzzing!
Server: Buzzing!
Server: Buzzing!
client connected
Buzz!
Buzz!
Buzz!
Buzz!
Buzz!
Buzz!
Buzz!
